In [ ]:
#importo las librerias que me van a servir para :
#requests para conectarnos a la API CoinGecko y obtener información actualizada
#pandas se utiliza para cargar y manipular los datos obtenidos de la API CoinGecko
#matplotlib.pyplot y seaborn para crear visualizaciones en el EDA
#locale en este caso para que los números y monedas se formateen de acuerdo con las convenciones en inglés (libreria de foro coingecko)

import requests
import locale
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Cargar los archivos CSV en DtaFrames separados para luego unirlos en uno solo df y guardarlo

In [ ]:
df_stable = pd.read_csv('SC_DATA.csv')
df_crypto = pd.read_csv('crypto_data.csv')

df_cryptocurrent = pd.concat([df_stable, df_crypto], ignore_index=True)

df_cryptocurrent.to_csv('cryptocurrency.csv', index=False)

In [ ]:
df_cryptocurrent.head(10)

In [ ]:
df_cryptocurrent.dtypes

In [ ]:
duplicates = df_stable.duplicated()
print(f"{duplicates.sum()}")

In [ ]:
df_stable.isnull().sum()

In [ ]:
#convierto date a datetime
df_cryptocurrent["Date"] = pd.to_datetime(df_cryptocurrent["Date"],  dayfirst=True)

In [ ]:
#establezco locale para que me formatee los numeros
#Definir una función para formatear números con el separador de miles.
#Aplico formato a las columnas numéricas, para mantener asi el precio en usd

locale.setlocale(locale.LC_ALL, 'es_ES.utf8')

def format_with_thousands_separator(number):
    return locale.format_string('%d', number, grouping=True)

numeric_columns = ['MarketCap', 'TotalVolume']

for column in numeric_columns:
    df_cryptocurrent[column] = df_cryptocurrent[column].apply(format_with_thousands_separator)

In [ ]:
#vemos los cambios
df_cryptocurrent.head(10)

In [ ]:
all_cryptocurrencies = df_cryptocurrent['Currency'].unique()

sample_cryptocurrencies = all_cryptocurrencies[:8]

df_sample = df_cryptocurrent[df_cryptocurrent['Currency'].isin(sample_cryptocurrencies)]

variable_combinations = [('Price', 'MarketCap'), ('Price', 'TotalVolume'), ('MarketCap', 'TotalVolume')]

plt.figure(figsize=(12, 8))
for var_x, var_y in variable_combinations:
    plt.subplot(2, 2, variable_combinations.index((var_x, var_y)) + 1)
    for crypto in sample_cryptocurrencies:
        subset = df_sample[df_sample['Currency'] == crypto]
        plt.scatter(subset[var_x], subset[var_y], label=crypto, alpha=0.6)
    plt.xlabel(var_x)
    plt.ylabel(var_y)
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.title(f'{var_x} vs {var_y}')

plt.tight_layout()
plt.show()

In [ ]:
#Gráfico de líneas con áreas: se utiliza para representar la evolución y las tendencias
# saltear el DataFrame para tener las fechas como índice y las cripto como columnas
pivot_df = df_cryptocurrent.pivot(index='Date', columns='Stablecoin', values='Price')

# Crear un gráfico de líneas con áreas para el precio de cada moneda en todas las fechas
plt.figure(figsize=(10, 6))
for crypto in pivot_df.columns:
    plt.fill_between(pivot_df.index, pivot_df[crypto], label=crypto, alpha=0.5)
    plt.plot(pivot_df.index, pivot_df[crypto], alpha=0.8)

plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.title('Evolución del Precio de las criptomonedas en todas las fechas')
plt.xticks(rotation=45)
plt.legend(title='Criptomoneda', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()


In [ ]:
#Matriz de correlación:
correlation_matrix = df_cryptocurrent.pivot_table(values='Price', columns='Stablecoin', index='Date')

plt.figure(figsize=(10, 8))
plt.imshow(correlation_matrix.corr(), cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.title('Matriz de Correlación entre Precios de Cryptocurrency')
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=45)
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.tight_layout()
plt.show()

In [ ]:
#Podemos observar de acuerdo a los colores correlaciones:

#Positivas: Las zonas de colores más cálidos (rojos) en la matriz indican correlaciones positivas entre las criptomonedas correspondientes. Esto significa que los precios de estas criptomonedas tienden a moverse en la misma dirección a lo largo del tiempo. Cuanto más intenso sea el tono de rojo, mayor será la correlación positiva entre las criptomonedas. Por ejemplo entre usd-coin, true-usd y dai., o entre BNB, SRP y Dogecoin.
#Negativas: Las áreas de colores más fríos (azules) indican correlaciones negativas entre las criptomonedas correspondientes. Esto significa que los precios de estas criptomonedas tienden a moverse en direcciones opuestas. Cuanto más intenso sea el tono de azul, mayor será la correlación negativa entre las criptomonedas. Por ejempo, paxos-standard con las demas.
#Bajas o Nulas: Las áreas de colores más neutros (tonos cercanos al blanco) sugieren correlaciones bajas o nulas entre las criptomonedas correspondientes. Esto indica que los precios de estas criptomonedas no están fuertemente relacionados entre sí y pueden moverse de manera independiente. Por ejemplo, usd-coin, dai y true-usd con binance-coin, FRAX y USDD.

In [ ]:
#guardo todo en un csv
df_cryptocurrent.to_csv('crypto_final.csv', index=False)